# MLflow Tracking - US Accidents

## 1. Configuration et Imports

In [3]:
pip install mlflow dagshub pandas numpy scikit-learn xgboost lightgbm catboost scipy python-dotenv 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Imports
import pandas as pd
import numpy as np
import pickle
import os
from datetime import datetime
from pathlib import Path

# MLflow
import mlflow
import mlflow.sklearn
import dagshub
from dotenv import load_dotenv

# ML Models
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

# Tuning
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score
)

import warnings
warnings.filterwarnings('ignore')

print("✅ Imports terminés")

✅ Imports terminés


In [5]:
# Configuration MLflow + DagsHub
load_dotenv()

DAGSHUB_USERNAME = os.getenv('DAGSHUB_USERNAME', 'smile.bradai')
DAGSHUB_TOKEN = os.getenv('DAGSHUB_TOKEN', '')
DAGSHUB_REPO = os.getenv('DAGSHUB_REPO_NAME', 'MLOPS-Project')

MLFLOW_TRACKING_URI = f"https://dagshub.com/{DAGSHUB_USERNAME}/{DAGSHUB_REPO}.mlflow"
EXPERIMENT_NAME = "US-Accidents"

# Configuration DagsHub
dagshub.init(repo_owner=DAGSHUB_USERNAME, repo_name=DAGSHUB_REPO, mlflow=True)

# Configuration MLflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

# Désactiver les features non supportées par DagsHub
os.environ['MLFLOW_ENABLE_LOGGED_MODEL_CREATION'] = 'false'

if DAGSHUB_TOKEN:
    os.environ['MLFLOW_TRACKING_USERNAME'] = DAGSHUB_USERNAME
    os.environ['MLFLOW_TRACKING_PASSWORD'] = DAGSHUB_TOKEN

print("✅ MLflow configuré")
print(f"📊 Tracking URI: {MLFLOW_TRACKING_URI}")
print(f"🧪 Experiment: {EXPERIMENT_NAME}")

Accessing as smile.bradai

Initialized MLflow to track repo "smile.bradai/MLOPS-Project"

Repository smile.bradai/MLOPS-Project initialized!

✅ MLflow configuré
📊 Tracking URI: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow
🧪 Experiment: US-Accidents


## 2. Chargement des Données

In [6]:
# Charger les données preprocessées
DATA_PATH = 'processors/preprocessed_data.pkl'

with open(DATA_PATH, 'rb') as f:
    data = pickle.load(f)

X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

print("✅ Données chargées")
print(f"   Train: {X_train.shape}")
print(f"   Test: {X_test.shape}")

✅ Données chargées
   Train: (602028, 20)
   Test: (60000, 20)


## 3. Fonctions Utilitaires

In [7]:
# Fonction pour calculer les métriques
# Fonction métriques adaptée pour multiclasse
def calculate_metrics(y_true, y_pred, y_proba):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted'),
        'recall': recall_score(y_true, y_pred, average='weighted'),
        'f1_score': f1_score(y_true, y_pred, average='weighted'),
        'roc_auc': roc_auc_score(y_true, y_proba, multi_class='ovr') if y_proba is not None else None
    }

# Fonction pour logger un modèle dans MLflow
def log_model_mlflow(model, model_name, stage, metrics, duration, best_params=None):
    """
    Log un modèle dans MLflow de manière compatible DagsHub
    """
    with mlflow.start_run(run_name=f"{model_name}_{stage}"):
        # Log params
        mlflow.log_param('model_name', model_name)
        mlflow.log_param('stage', stage)
        mlflow.log_param('n_features', X_train.shape[1])
        
        # Log best params si disponibles
        if best_params:
            for k, v in best_params.items():
                try:
                    mlflow.log_param(f'best_{k}', v)
                except:
                    pass
        
        # Log metrics
        for metric_name, metric_value in metrics.items():
            mlflow.log_metric(metric_name, metric_value)
        mlflow.log_metric('training_duration', duration)
        
        # Sauvegarder le modèle localement
        model_filename = f"{model_name}_{stage}.pkl"
        with open(model_filename, 'wb') as f:
            pickle.dump(model, f)
        
        # Log comme artifact
        try:
            mlflow.log_artifact(model_filename)
        except:
            pass
        
        run_id = mlflow.active_run().info.run_id
        return run_id, model_filename

print("✅ Fonctions utilitaires définies")

✅ Fonctions utilitaires définies


## 4. Entraînement des Modèles Baseline (4 modèles)

In [8]:
from sklearn.preprocessing import LabelEncoder

# =====================================================================================
# 1️⃣ ENCODAGE DE LA TARGET (MULTICLASS OBLIGATOIRE POUR XGBOOST)
# =====================================================================================
print("="*80)
print("ENCODAGE DE LA VARIABLE CIBLE")
print("="*80)

label_encoder = LabelEncoder()

y_train_enc = label_encoder.fit_transform(y_train)
y_test_enc = label_encoder.transform(y_test)

print("Classes originales :", np.unique(y_train))
print("Classes encodées    :", np.unique(y_train_enc))

ENCODAGE DE LA VARIABLE CIBLE
Classes originales : [1 2 3 4]
Classes encodées    : [0 1 2 3]


In [9]:
# Définir les modèles baseline
baseline_models = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'XGBoost': XGBClassifier(n_estimators=100, random_state=42, n_jobs=-1, eval_metric='logloss'),
    'LightGBM': LGBMClassifier(n_estimators=100, random_state=42, n_jobs=-1, verbose=-1),
    'CatBoost': CatBoostClassifier(iterations=100, random_state=42, verbose=0)
}

baseline_results = []
trained_models = {}

print("🚀 Entraînement des modèles BASELINE...\n")

for name, model in baseline_models.items():
    print(f"📊 {name}...", end=" ")
    start = datetime.now()
    
    # Entraînement
    model.fit(X_train, y_train_enc)
    
    # Prédictions
    y_pred = model.predict(X_test)
    
    # Pour ROC-AUC multiclasse, garder toutes les colonnes
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)
    else:
        # Pour les modèles qui n'ont pas predict_proba
        y_proba = None
    
    # Métriques
    metrics = calculate_metrics(y_test_enc, y_pred, y_proba)
    duration = (datetime.now() - start).total_seconds()
    
    # Log dans MLflow
    run_id, model_file = log_model_mlflow(model, name, 'baseline', metrics, duration)
    
    # Stocker
    trained_models[f"{name}_baseline"] = model
    baseline_results.append({
        'model': name,
        'stage': 'baseline',
        'run_id': run_id,
        **metrics,
        'duration': duration
    })
    
    print(f"ROC-AUC: {metrics['roc_auc']:.4f} ({duration:.1f}s)")

print("\n✅ Baseline terminé!")

🚀 Entraînement des modèles BASELINE...

📊 RandomForest... 🏃 View run RandomForest_baseline at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0/runs/bd107139f88d4d6094da2ec134582d1b
🧪 View experiment at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0
ROC-AUC: 0.8189 (58.5s)
📊 XGBoost... 🏃 View run XGBoost_baseline at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0/runs/e1ba0ddf2d8b43698d69bb0b131b302a
🧪 View experiment at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0
ROC-AUC: 0.8327 (16.3s)
📊 LightGBM... 🏃 View run LightGBM_baseline at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0/runs/4738e08247d84db29349bac05b6e6669
🧪 View experiment at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0
ROC-AUC: 0.8344 (14.3s)
📊 CatBoost... 🏃 View run CatBoost_baseline at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0/runs/f68e870ad5624731999

## 5. Fine-Tuning (4 modèles avec n_iter=5)

In [10]:
# Grilles de recherche simplifiées
param_distributions = {
    'RandomForest': {
        'n_estimators': randint(100, 300),
        'max_depth': randint(10, 25)
    },
    'XGBoost': {
        'n_estimators': randint(100, 300),
        'max_depth': randint(3, 8),
        'learning_rate': uniform(0.01, 0.2)
    },
    'LightGBM': {
        'n_estimators': randint(100, 300),
        'max_depth': randint(3, 8),
        'learning_rate': uniform(0.01, 0.2)
    },
    'CatBoost': {
        'iterations': randint(100, 300),
        'depth': randint(4, 8),
        'learning_rate': uniform(0.01, 0.2)
    }
}

tuned_results = []
N_ITER = 5  # Nombre d'itérations
CV_FOLDS = 3

print(f"🔍 Fine-Tuning ({N_ITER} iterations × {CV_FOLDS} folds)...\n")

for name, base_model in baseline_models.items():
    print(f"📊 {name}...", end=" ")
    start = datetime.now()
    
    # RandomizedSearchCV
    search = RandomizedSearchCV(
        base_model,
        param_distributions[name],
        n_iter=N_ITER,
        cv=CV_FOLDS,
        scoring='roc_auc_ovr',
        n_jobs=-1,
        random_state=42,
        verbose=0
    )
    
    search.fit(X_train, y_train_enc)
    best_model = search.best_estimator_
    
    # Prédictions
    y_pred = best_model.predict(X_test)
    
    # Pour ROC-AUC multiclasse, garder toutes les colonnes
    if hasattr(best_model, "predict_proba"):
        y_proba = best_model.predict_proba(X_test)  # <- garder toutes les classes
    else:
        y_proba = None
    
    # Métriques
    metrics = calculate_metrics(y_test_enc, y_pred, y_proba)
    duration = (datetime.now() - start).total_seconds()
    
    # Log dans MLflow
    run_id, model_file = log_model_mlflow(
        best_model, name, 'tuned', metrics, duration, search.best_params_
    )
    
    # Stocker
    trained_models[f"{name}_tuned"] = best_model
    tuned_results.append({
        'model': name,
        'stage': 'tuned',
        'run_id': run_id,
        **metrics,
        'duration': duration
    })
    
    print(f"ROC-AUC: {metrics['roc_auc']:.4f} ({duration:.1f}s)")

print("\n✅ Fine-tuning terminé!")

🔍 Fine-Tuning (5 iterations × 3 folds)...

📊 RandomForest... 🏃 View run RandomForest_tuned at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0/runs/610ba93bb8514d26bc25cba27ae2ce23
🧪 View experiment at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0
ROC-AUC: 0.8303 (914.7s)
📊 XGBoost... 🏃 View run XGBoost_tuned at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0/runs/aa8853bb079f4752b92c00d17b77b950
🧪 View experiment at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0
ROC-AUC: 0.8355 (252.2s)
📊 LightGBM... 🏃 View run LightGBM_tuned at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0/runs/afffe80981b447669909d290079dd5b3
🧪 View experiment at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0
ROC-AUC: 0.8398 (258.4s)
📊 CatBoost... 🏃 View run CatBoost_tuned at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0/runs/f94f4297c24f497a8bed2001a

## 6. Stacking Ensembles (2 modèles)

In [11]:
# Prendre les meilleurs modèles tunés pour le stacking
estimators = [
    ('rf', trained_models['RandomForest_tuned']),
    ('xgb', trained_models['XGBoost_tuned']),
    ('lgbm', trained_models['LightGBM_tuned']),
    ('cat', trained_models['CatBoost_tuned'])
]

ensemble_results = []

# 1. Stacking avec Logistic Regression
print("📊 Stacking (LogReg)...", end=" ")
start = datetime.now()

stacking_lr = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(random_state=42, max_iter=1000),
    cv=3,
    n_jobs=-1
)

stacking_lr.fit(X_train, y_train_enc)
y_pred = stacking_lr.predict(X_test)

# ROC-AUC multiclasse
if hasattr(stacking_lr, "predict_proba"):
    y_proba = stacking_lr.predict_proba(X_test)  # garder toutes les classes
else:
    y_proba = None

metrics_stack_lr = calculate_metrics(y_test_enc, y_pred, y_proba)
duration = (datetime.now() - start).total_seconds()

run_id_lr, _ = log_model_mlflow(stacking_lr, 'Stacking_LR', 'ensemble', metrics_stack_lr, duration)
trained_models['Stacking_LR'] = stacking_lr
ensemble_results.append({
    'model': 'Stacking_LR',
    'stage': 'ensemble',
    'run_id': run_id_lr,
    **metrics_stack_lr,
    'duration': duration
})

print(f"ROC-AUC: {metrics_stack_lr['roc_auc']:.4f} ({duration:.1f}s)")

# 2. Voting Classifier (soft voting)
print("📊 Voting (Soft)...", end=" ")
start = datetime.now()

voting_clf = VotingClassifier(
    estimators=estimators,
    voting='soft',
    n_jobs=-1
)

voting_clf.fit(X_train, y_train_enc)
y_pred = voting_clf.predict(X_test)

if hasattr(voting_clf, "predict_proba"):
    y_proba = voting_clf.predict_proba(X_test)  # garder toutes les classes
else:
    y_proba = None

metrics_voting = calculate_metrics(y_test_enc, y_pred, y_proba)
duration = (datetime.now() - start).total_seconds()

run_id_vote, _ = log_model_mlflow(voting_clf, 'Voting_Soft', 'ensemble', metrics_voting, duration)
trained_models['Voting_Soft'] = voting_clf
ensemble_results.append({
    'model': 'Voting_Soft',
    'stage': 'ensemble',
    'run_id': run_id_vote,
    **metrics_voting,
    'duration': duration
})

print(f"ROC-AUC: {metrics_voting['roc_auc']:.4f} ({duration:.1f}s)")

print("\n✅ Ensembles terminés!")

📊 Stacking (LogReg)... 🏃 View run Stacking_LR_ensemble at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0/runs/aad106fa55994722894f48083dc4467f
🧪 View experiment at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0
ROC-AUC: 0.7769 (546.1s)
📊 Voting (Soft)... 🏃 View run Voting_Soft_ensemble at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0/runs/7685da7408c14aefa7c609a693afb259
🧪 View experiment at: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow/#/experiments/0
ROC-AUC: 0.8506 (225.8s)

✅ Ensembles terminés!


## 7. Lecture des Résultats avec Pandas DataFrame

In [12]:
# Combiner tous les résultats
all_results = baseline_results + tuned_results + ensemble_results
df_results = pd.DataFrame(all_results)

print("📊 Résultats de tous les modèles:\n")
print(df_results[['model', 'stage', 'roc_auc', 'f1_score', 'duration']].to_string(index=False))

# Afficher le top 5 par ROC-AUC
print("\n🏆 Top 5 modèles (ROC-AUC):\n")
top5 = df_results.nlargest(5, 'roc_auc')[['model', 'stage', 'roc_auc', 'f1_score']]
print(top5.to_string(index=False))

📊 Résultats de tous les modèles:

       model    stage  roc_auc  f1_score   duration
RandomForest baseline 0.818939  0.923067  58.464439
     XGBoost baseline 0.832728  0.899380  16.314418
    LightGBM baseline 0.834402  0.878512  14.320970
    CatBoost baseline 0.834785  0.877157  21.504200
RandomForest    tuned 0.830311  0.920292 914.681771
     XGBoost    tuned 0.835474  0.901055 252.225920
    LightGBM    tuned 0.839839  0.897669 258.414786
    CatBoost    tuned 0.840556  0.883743 487.847914
 Stacking_LR ensemble 0.776920  0.920898 546.096706
 Voting_Soft ensemble 0.850637  0.905678 225.777454

🏆 Top 5 modèles (ROC-AUC):

      model    stage  roc_auc  f1_score
Voting_Soft ensemble 0.850637  0.905678
   CatBoost    tuned 0.840556  0.883743
   LightGBM    tuned 0.839839  0.897669
    XGBoost    tuned 0.835474  0.901055
   CatBoost baseline 0.834785  0.877157


In [13]:
# Lire depuis MLflow directement
print("\n📥 Lecture depuis MLflow...\n")

# Obtenir l'ID de l'experiment
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
experiment_id = experiment.experiment_id

# Rechercher toutes les runs
df_mlflow = mlflow.search_runs(
    experiment_ids=[experiment_id],
    filter_string="metrics.roc_auc > 0",
    order_by=["metrics.roc_auc DESC"]
)

# Afficher les colonnes importantes
if len(df_mlflow) > 0:
    cols_to_show = ['run_id', 'params.model_name', 'params.stage', 
                    'metrics.roc_auc', 'metrics.f1_score', 'metrics.training_duration']
    available_cols = [col for col in cols_to_show if col in df_mlflow.columns]
    print(df_mlflow[available_cols].head(10))
    print(f"\n✅ {len(df_mlflow)} runs trouvées dans MLflow")
else:
    print("⚠️  Aucune run trouvée dans MLflow")


📥 Lecture depuis MLflow...

                             run_id params.model_name params.stage  \
0  7685da7408c14aefa7c609a693afb259       Voting_Soft     ensemble   
1  820cffda582e499aa4816c3c604f008a       Voting_Soft     ensemble   
2  f94f4297c24f497a8bed2001a21602f3          CatBoost        tuned   
3  e003540795f74370a584ba99de0dea0d          CatBoost        tuned   
4  923676de0a9444d0a8802ca79c75dcbc          CatBoost        tuned   
5  2fdf0d285dd44bf9be8cf2ad149fa049          CatBoost        tuned   
6  afffe80981b447669909d290079dd5b3          LightGBM        tuned   
7  d7085530057f4f918098460a29bedb1c          LightGBM        tuned   
8  f99d9bf50f0a423ab20f1900a755374e          LightGBM        tuned   
9  2fac1346cc2247119871ea21e4e236be          LightGBM        tuned   

   metrics.roc_auc  metrics.f1_score  metrics.training_duration  
0         0.850637          0.905678                 225.777454  
1         0.850637          0.905678                 213.799137  
2 

## 8. Sélection du Meilleur Modèle (ROC-AUC)

In [14]:
# Depuis notre DataFrame local
best_idx = df_results['roc_auc'].idxmax()
best_row = df_results.loc[best_idx]

best_model_name = best_row['model']
best_stage = best_row['stage']
best_run_id = best_row['run_id']
best_roc_auc = best_row['roc_auc']

print("🏆 MEILLEUR MODÈLE (ROC-AUC)")
print("="*60)
print(f"Modèle:    {best_model_name}")
print(f"Stage:     {best_stage}")
print(f"ROC-AUC:   {best_roc_auc:.4f}")
print(f"F1-Score:  {best_row['f1_score']:.4f}")
print(f"Precision: {best_row['precision']:.4f}")
print(f"Recall:    {best_row['recall']:.4f}")
print(f"Run ID:    {best_run_id}")
print("="*60)

# Récupérer le modèle
best_model_key = f"{best_model_name}_{best_stage}" if best_stage != 'ensemble' else best_model_name
best_model = trained_models.get(best_model_key)

print(f"\n✅ Modèle chargé : {type(best_model).__name__}")

🏆 MEILLEUR MODÈLE (ROC-AUC)
Modèle:    Voting_Soft
Stage:     ensemble
ROC-AUC:   0.8506
F1-Score:  0.9057
Precision: 0.9056
Recall:    0.9062
Run ID:    7685da7408c14aefa7c609a693afb259

✅ Modèle chargé : VotingClassifier


## 9. Chargement du Modèle depuis MLflow

In [17]:
# Charger le modèle depuis le run_id
print(f"📥 Chargement du modèle depuis run_id: {best_run_id}\n")

# Via artifact (compatible DagsHub)
try:
    # Télécharger l'artifact
    model_filename = f"{best_model_name}_{best_stage}.pkl"
    artifact_uri = f"runs:/{best_run_id}/{model_filename}"  # ✅ chemin corrigé
    
    local_path = mlflow.artifacts.download_artifacts(artifact_uri)
    
    # Charger avec pickle
    with open(local_path, 'rb') as f:
        loaded_model = pickle.load(f)
    
    print(f"✅ Modèle chargé depuis MLflow artifact")
    print(f"   Type: {type(loaded_model).__name__}")
    
except Exception as e:
    print(f"⚠️  Erreur de chargement depuis MLflow: {e}")
    print(f"   Utilisation du modèle en mémoire à la place")
    loaded_model = best_model

# Alternative: Charger depuis fichier local
local_model_file = f"{best_model_name}_{best_stage}.pkl"
if os.path.exists(local_model_file):
    with open(local_model_file, 'rb') as f:
        loaded_model_local = pickle.load(f)
    print(f"✅ Modèle également disponible localement: {local_model_file}")


📥 Chargement du modèle depuis run_id: 7685da7408c14aefa7c609a693afb259



✅ Modèle chargé depuis MLflow artifact
   Type: VotingClassifier
✅ Modèle également disponible localement: Voting_Soft_ensemble.pkl


In [19]:
# Test du modèle chargé
print("\n🧪 Test du modèle chargé...\n")

# =========================
# Prédictions
# =========================
y_pred_loaded = loaded_model.predict(X_test)

# Probabilités (multiclass)
y_proba_loaded = loaded_model.predict_proba(X_test)

# =========================
# Métriques adaptées multiclass
# =========================
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score
)

test_metrics = {
    "Accuracy": accuracy_score(y_test_enc, y_pred_loaded),
    "Precision": precision_score(y_test_enc, y_pred_loaded, average="macro"),
    "Recall": recall_score(y_test_enc, y_pred_loaded, average="macro"),
    "F1-Score": f1_score(y_test_enc, y_pred_loaded, average="macro"),
    "ROC-AUC": roc_auc_score(
        y_test_enc,
        y_proba_loaded,
        multi_class="ovr",
        average="macro"
    )
}

# =========================
# Affichage des résultats
# =========================
print("📊 Performances du modèle chargé:")
for metric, value in test_metrics.items():
    print(f"   {metric:12s}: {value:.4f}")

# =========================
# Test sur quelques exemples
# =========================
print("\n🔍 Prédictions sur 5 exemples:")

sample_predictions = loaded_model.predict(X_test[:5])
sample_probas = loaded_model.predict_proba(X_test[:5])

for i in range(5):
    print(f"\n   Sample {i+1}:")
    print(f"     Severity prédite : {sample_predictions[i]}")
    print(f"     Probabilités par classe :")

    for cls, prob in enumerate(sample_probas[i]):
        print(f"       Classe {cls} → {prob:.4f}")

print("\n✅ Modèle multiclasses fonctionne correctement!")



🧪 Test du modèle chargé...

📊 Performances du modèle chargé:
   Accuracy    : 0.9062
   Precision   : 0.4951
   Recall      : 0.5091
   F1-Score    : 0.5005
   ROC-AUC     : 0.8506

🔍 Prédictions sur 5 exemples:

   Sample 1:
     Severity prédite : 2
     Probabilités par classe :
       Classe 0 → 0.0010
       Classe 1 → 0.4739
       Classe 2 → 0.5226
       Classe 3 → 0.0025

   Sample 2:
     Severity prédite : 2
     Probabilités par classe :
       Classe 0 → 0.0021
       Classe 1 → 0.1350
       Classe 2 → 0.8627
       Classe 3 → 0.0002

   Sample 3:
     Severity prédite : 1
     Probabilités par classe :
       Classe 0 → 0.0265
       Classe 1 → 0.9561
       Classe 2 → 0.0174
       Classe 3 → 0.0000

   Sample 4:
     Severity prédite : 1
     Probabilités par classe :
       Classe 0 → 0.0126
       Classe 1 → 0.9107
       Classe 2 → 0.0753
       Classe 3 → 0.0015

   Sample 5:
     Severity prédite : 2
     Probabilités par classe :
       Classe 0 → 0.0014
       

## 10. Enregistrement dans Model Registry (Local)

In [20]:
# Créer un Model Registry local
MODEL_REGISTRY_DIR = Path("model_registry")
MODEL_REGISTRY_DIR.mkdir(exist_ok=True)

def register_model(model, model_name, version="1.0.0", stage="production"):
    """
    Enregistre un modèle dans le registry local
    """
    import json
    import shutil
    
    # Créer la structure
    model_dir = MODEL_REGISTRY_DIR / model_name.replace(" ", "_")
    model_dir.mkdir(exist_ok=True)
    
    version_dir = model_dir / version
    version_dir.mkdir(exist_ok=True)
    
    # Sauvegarder le modèle
    model_path = version_dir / "model.pkl"
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    
    # Métadonnées
    metadata = {
        "model_name": model_name,
        "version": version,
        "stage": stage,
        "registered_at": datetime.now().isoformat(),
        "metrics": test_metrics,
        "run_id": best_run_id
    }
    
    with open(version_dir / "metadata.json", 'w') as f:
        json.dump(metadata, f, indent=2)
    
    # Lien production
    if stage == "production":
        prod_path = model_dir / "production.pkl"
        shutil.copy(model_path, prod_path)
    
    return str(model_path)

# Enregistrer le meilleur modèle
print("📦 Enregistrement dans Model Registry...\n")

registry_name = f"Best_Fraud_{best_model_name}"
model_path = register_model(
    model=loaded_model,
    model_name=registry_name,
    version="1.0.0",
    stage="production"
)

print(f"✅ Modèle enregistré dans le registry")
print(f"   Nom: {registry_name}")
print(f"   Version: 1.0.0")
print(f"   Stage: production")
print(f"   Path: {model_path}")

📦 Enregistrement dans Model Registry...

✅ Modèle enregistré dans le registry
   Nom: Best_Fraud_Voting_Soft
   Version: 1.0.0
   Stage: production
   Path: model_registry\Best_Fraud_Voting_Soft\1.0.0\model.pkl


In [21]:
# Fonction pour charger depuis le registry
def load_from_registry(model_name, stage="production"):
    """Charge un modèle depuis le registry local"""
    import json
    
    model_dir = MODEL_REGISTRY_DIR / model_name.replace(" ", "_")
    model_path = model_dir / f"{stage}.pkl"
    
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    
    # Charger les métadonnées
    versions = [d for d in model_dir.iterdir() if d.is_dir()]
    if versions:
        latest_version = sorted(versions)[-1]
        with open(latest_version / "metadata.json", 'r') as f:
            metadata = json.load(f)
    else:
        metadata = {}
    
    return model, metadata

# Test du chargement
print("\n🔄 Test de chargement depuis le registry...\n")

loaded_from_registry, metadata = load_from_registry(registry_name, stage="production")

print(f"✅ Modèle chargé depuis le registry")
print(f"   Nom: {metadata.get('model_name', 'N/A')}")
print(f"   Version: {metadata.get('version', 'N/A')}")
print(f"   ROC-AUC: {metadata.get('metrics', {}).get('roc_auc', 0):.4f}")

# Test de prédiction
test_pred = loaded_from_registry.predict(X_test[:5])
print(f"\n🧪 Test de prédiction: {test_pred}")
print("✅ Le modèle fonctionne correctement!")


🔄 Test de chargement depuis le registry...

✅ Modèle chargé depuis le registry
   Nom: Best_Fraud_Voting_Soft
   Version: 1.0.0
   ROC-AUC: 0.0000

🧪 Test de prédiction: [2 2 1 1 2]
✅ Le modèle fonctionne correctement!


## 11. Résumé Final

In [22]:
print("\n" + "="*80)
print("🎉 RÉSUMÉ FINAL - MLflow Tracking")
print("="*80)

print(f"\n📊 Modèles entraînés:")
print(f"   • Baseline:  4 modèles")
print(f"   • Tuned:     4 modèles (n_iter={N_ITER})")
print(f"   • Ensemble:  2 modèles (Stacking + Voting)")
print(f"   • TOTAL:     10 modèles")

print(f"\n🏆 Meilleur modèle:")
print(f"   • Nom:       {best_model_name}")
print(f"   • Stage:     {best_stage}")
print(f"   • ROC-AUC:   {best_roc_auc:.4f}")
print(f"   • F1-Score:  {best_row['f1_score']:.4f}")

print(f"\n🔗 MLflow:")
print(f"   • Tracking URI: {MLFLOW_TRACKING_URI}")
print(f"   • Experiment:   {EXPERIMENT_NAME}")
print(f"   • Runs totales: {len(df_results)}")

print(f"\n📦 Model Registry:")
print(f"   • Nom:     {registry_name}")
print(f"   • Version: 1.0.0")
print(f"   • Stage:   production")
print(f"   • Path:    {MODEL_REGISTRY_DIR / registry_name.replace(' ', '_')}")

print("\n" + "="*80)
print("✅ Pipeline MLflow terminé avec succès!")
print("="*80)

print("\n💡 Prochaines étapes:")
print("   1. Consultez MLflow UI pour voir toutes les runs")
print("   2. Chargez le modèle avec: load_from_registry()")
print("   3. Déployez en production")
print("   4. Configurez le monitoring")


🎉 RÉSUMÉ FINAL - MLflow Tracking

📊 Modèles entraînés:
   • Baseline:  4 modèles
   • Tuned:     4 modèles (n_iter=5)
   • Ensemble:  2 modèles (Stacking + Voting)
   • TOTAL:     10 modèles

🏆 Meilleur modèle:
   • Nom:       Voting_Soft
   • Stage:     ensemble
   • ROC-AUC:   0.8506
   • F1-Score:  0.9057

🔗 MLflow:
   • Tracking URI: https://dagshub.com/smile.bradai/MLOPS-Project.mlflow
   • Experiment:   US-Accidents
   • Runs totales: 10

📦 Model Registry:
   • Nom:     Best_Fraud_Voting_Soft
   • Version: 1.0.0
   • Stage:   production
   • Path:    model_registry\Best_Fraud_Voting_Soft

✅ Pipeline MLflow terminé avec succès!

💡 Prochaines étapes:
   1. Consultez MLflow UI pour voir toutes les runs
   2. Chargez le modèle avec: load_from_registry()
   3. Déployez en production
   4. Configurez le monitoring
